In [1]:
# notebooks/analysis.ipynb

# The magic command for interactive plots
%matplotlib notebook

import torch
import torch.nn as nn
import matplotlib.pyplot as plt

# --- NEW, CLEANER IMPORTS ---
from rallyrobopilot.ml.model import RobopilotMLP
from rallyrobopilot.ml.training import prepare_data
from rallyrobopilot.ml.utils import LossPlotter

# --- Set file paths correctly ---
MODEL_SAVE_PATH = "../models/robopilot_notebook_run.pth"
FIGURE_SAVE_PATH = "../reports/figures/loss_notebook_run.png"

package_folder: /home/rouge/machine_learning_course/week-5/RallyRobotPilot_2025/.venv/lib/python3.12/site-packages/ursina
asset_folder: /home/rouge/machine_learning_course/week-5/RallyRobotPilot_2025/.venv/lib/python3.12/site-packages


In [2]:
# --- 1. Hyperparameters and Setup ---
NUM_EPOCHS = 25
BATCH_SIZE = 64
LEARNING_RATE = 0.001
MODEL_SAVE_PATH = "robopilot_model_v1.pth"

# --- 2. Data Preparation ---
# Call your imported function to get the data
# This is interactive - you can easily change the sources and re-run just this cell
train_loader, val_loader = prepare_data(
    batch_size=BATCH_SIZE, 
    sources=['normal', 'mirrored']
)

--- Starting Data Preparation for sources: ['normal', 'mirrored'] ---
[!] Warning: No data found in 'data/normal'
[!] Warning: No data found in 'data/mirrored'

[!] No data was loaded! Check the 'sources' argument and your folder structure.


In [3]:
if train_loader:
    # --- 3. Model, Loss, and Optimizer Initialization ---
    
    # --- CORRECTED LINE ---
    # Instantiate the model with your desired dropout rate.
    model = RobopilotMLP(dropout_rate=0.5)

    # --- REMOVED LINE ---
    # We no longer need this line: model.layers.add_module("dropout_1", nn.Dropout(0.5))

    criterion = nn.BCELoss()
    # Add weight_decay to the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
    
    plotter = LossPlotter()

    # --- 4. The Training Loop ---
    # (You can copy the training loop here or also put it in a function in train.py)
    print("\n--- Starting Model Training ---")
    for epoch in range(NUM_EPOCHS):
        model.train()
        total_train_loss = 0
        for i, (features, labels) in enumerate(train_loader):
            outputs = model(features)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        avg_train_loss = total_train_loss / len(train_loader)

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for features, labels in val_loader:
                outputs = model(features)
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()
        avg_val_loss = total_val_loss / len(val_loader)
        
        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')
        plotter.update(avg_train_loss, avg_val_loss)

    print("--- Model Training Complete ---")
    torch.save(model.state_dict(), MODEL_SAVE_PATH)
    plotter.save(FIGURE_SAVE_PATH)

In [1]:
!python ../scripts/run_autopilot.py ../models/robopilot_model_v2.pth

package_folder: /home/rouge/machine_learning_course/week-5/RallyRobotPilot_2025/.venv/lib/python3.12/site-packages/ursina
asset_folder: ../scripts
Initializing Autopilot Controller...
✅ Model loaded successfully from ../models/robopilot_model_v2.pth
🤖 Autopilot Activated.
🛑 Autopilot Deactivated.
